In [1]:
# -*- coding: utf-8 -*-
"""
% Energy in the Built Environment
% Assignment 2: Optimal Home Energy Management
% Dr. Tarek AlSkaif

"""
import gurobipy as gp
import csv
import pandas as pd #for csv reading
import numpy as np 
import matplotlib.pyplot as plt #for plotting

"""
Import your input data for the model
"""
summer = pd.read_csv("AssB_Input_Group4_summer.csv")
winter = pd.read_csv("AssB_Input_Group4_winter.csv")

    # dynamic electricity prices vector
    #household's 15-min PV generation vector
    #household's 15-min demand vector


#for now setting up just for summer as thinking when we make it a function can specify summer or winter
Ppv = summer['PV generation [kW]']
Pdem = summer['Residential load [kW]']
Celec = summer['Electricity price [euro/kWh]']

"""
Parameters value
"""
######## Time-step
Delta_t = 0.25 # 15 minute (0.25 hour) intervals
T=int(24*3*1/Delta_t) #number of time-slots (in three days)

######## Limits on grid and max, min, and initial SOC
Pgridmax = 3 #[kW]
Pbatmax = 4 #[kW]

SoC_min = 0.2 #[-] (battery min state of charge)
SoC_max = 1 #[-] (battery max state of charge)
SoC0 = 0.5 #[-] (initial battery state of charge at the beginning of the day)

C_bat = 13.5 #battery capacity parameter for a Tesla Powerwall rated at 13,5 [kWh]
eff_dis = 0.94 #battery discharging efficeicny
eff_ch = 0.94 #battery charging efficeicny

######## Plot power demand and PV generation data
f1 = plt.figure(1)

######## other parameters too add?


"""
Step 1: Create a model
"""
m=gp.Model()

"""
Step 2: Define variables
"""
######## Define your decision variables for the time horizon using addVars

# note one parameter is obj = , we have not set it, not sure what it does
Pbat_ch = m.addVars(T, lb= 0, ub= Pbatmax, vtype= gp.GRB.CONTINUOUS, name= "Pbat_ch")
Pbat_dis = m.addVars(T, lb= 0, ub= Pbatmax, vtype= gp.GRB.CONTINUOUS, name= "Pbat_dis")

Pgrid = m.addVars(T, lb= -Pgridmax, ub= Pgridmax, vtype= gp.GRB.CONTINUOUS, name= "Pgrid")

SoC = m.addVars(T, lb= SoC_min, ub= SoC_max, vtype= gp.GRB.CONTINUOUS, name= "SoC")


######## Nonnegative variables - not required since specified in upper/lower bounds of variable definitions

    
"""
Step 3: Add constraints
"""
######## Nonnegative variables - not required since specified in upper/lower bounds of variable definitions
   
######## Power balance formula
m.addConstrs(Pgrid[t] + Ppv[t] + Pbat_dis[t] - Pbat_ch[t] == Pdem[t] for t in range(T))
        
######## Battery SoC dynamics constraint 
SoC[0] == SoC0
m.addConstrs(SoC[t] == SoC[t-1] + ((Pbat_ch[t]*Delta_t*eff_ch)/C_bat) - (Pbat_dis[t]*Delta_t)/(eff_dis*C_bat) for t in range(1,T))


######## SoC constraints 
m.addConstrs(SoC_min <= SoC[t] for t in range(T))
m.addConstrs(SoC_max >= SoC[t] for t in range(T))

######## Power boundaries - not required since specified in upper/lower bounds of variable definitions

    
"""
Step 4: Set objective function
"""

obj = gp.quicksum(Celec[t]*Pgrid[t] for t in range(T))
m.setObjective(obj, gp.GRB.MINIMIZE)


"""
Step 5: Solve model
"""
m.optimize()

Academic license - for non-commercial use only - expires 2022-10-02
Using license file C:\Users\NCG\gurobi.lic
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1151 rows, 1152 columns and 2588 nonzeros
Model fingerprint: 0x6807132b
Coefficient statistics:
  Matrix range     [2e-02, 1e+00]
  Objective range  [3e-02, 7e-02]
  Bounds range     [2e-01, 4e+00]
  RHS range        [2e-03, 1e+00]
Presolve removed 577 rows and 3 columns
Presolve time: 0.03s
Presolved: 574 rows, 1149 columns, 2009 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -4.1997720e+01   4.579598e+02   0.000000e+00      0s
     869   -3.3473534e+00   0.000000e+00   0.000000e+00      0s

Solved in 869 iterations and 0.04 seconds
Optimal objective -3.347353407e+00


<Figure size 432x288 with 0 Axes>

In [2]:
SoC

{0: <gurobi.Var SoC[0] (value 1.0)>,
 1: <gurobi.Var SoC[1] (value 0.9340059101654845)>,
 2: <gurobi.Var SoC[2] (value 0.8683372734436563)>,
 3: <gurobi.Var SoC[3] (value 0.8029720252167061)>,
 4: <gurobi.Var SoC[4] (value 0.8029720252167061)>,
 5: <gurobi.Var SoC[5] (value 0.8029720252167061)>,
 6: <gurobi.Var SoC[6] (value 0.8029720252167061)>,
 7: <gurobi.Var SoC[7] (value 0.8029720252167061)>,
 8: <gurobi.Var SoC[8] (value 0.8029720252167061)>,
 9: <gurobi.Var SoC[9] (value 0.8029720252167061)>,
 10: <gurobi.Var SoC[10] (value 0.8029720252167061)>,
 11: <gurobi.Var SoC[11] (value 0.8029720252167061)>,
 12: <gurobi.Var SoC[12] (value 0.8029720252167061)>,
 13: <gurobi.Var SoC[13] (value 0.8029720252167061)>,
 14: <gurobi.Var SoC[14] (value 0.8029720252167061)>,
 15: <gurobi.Var SoC[15] (value 0.8029720252167061)>,
 16: <gurobi.Var SoC[16] (value 0.8029720252167061)>,
 17: <gurobi.Var SoC[17] (value 0.8029720252167061)>,
 18: <gurobi.Var SoC[18] (value 0.8029720252167061)>,
 19: <gur

In [4]:
Pbat_ch

{0: <gurobi.Var Pbat_ch[0] (value 0.0)>,
 1: <gurobi.Var Pbat_ch[1] (value 0.0)>,
 2: <gurobi.Var Pbat_ch[2] (value 0.0)>,
 3: <gurobi.Var Pbat_ch[3] (value 0.0)>,
 4: <gurobi.Var Pbat_ch[4] (value 0.0)>,
 5: <gurobi.Var Pbat_ch[5] (value 0.0)>,
 6: <gurobi.Var Pbat_ch[6] (value 0.0)>,
 7: <gurobi.Var Pbat_ch[7] (value 0.0)>,
 8: <gurobi.Var Pbat_ch[8] (value 0.0)>,
 9: <gurobi.Var Pbat_ch[9] (value 0.0)>,
 10: <gurobi.Var Pbat_ch[10] (value 0.0)>,
 11: <gurobi.Var Pbat_ch[11] (value 0.0)>,
 12: <gurobi.Var Pbat_ch[12] (value 0.0)>,
 13: <gurobi.Var Pbat_ch[13] (value 0.0)>,
 14: <gurobi.Var Pbat_ch[14] (value 0.0)>,
 15: <gurobi.Var Pbat_ch[15] (value 0.0)>,
 16: <gurobi.Var Pbat_ch[16] (value 0.0)>,
 17: <gurobi.Var Pbat_ch[17] (value 0.0)>,
 18: <gurobi.Var Pbat_ch[18] (value 0.0)>,
 19: <gurobi.Var Pbat_ch[19] (value 0.0)>,
 20: <gurobi.Var Pbat_ch[20] (value 0.0)>,
 21: <gurobi.Var Pbat_ch[21] (value 0.0)>,
 22: <gurobi.Var Pbat_ch[22] (value 0.0)>,
 23: <gurobi.Var Pbat_ch[23] (v

In [9]:
Pbat_dis

{0: <gurobi.Var Pbat_dis[0] (value 3.36722)>,
 1: <gurobi.Var Pbat_dis[1] (value 3.34986)>,
 2: <gurobi.Var Pbat_dis[2] (value 3.33334)>,
 3: <gurobi.Var Pbat_dis[3] (value 3.31794)>,
 4: <gurobi.Var Pbat_dis[4] (value 0.0)>,
 5: <gurobi.Var Pbat_dis[5] (value 0.0)>,
 6: <gurobi.Var Pbat_dis[6] (value 0.0)>,
 7: <gurobi.Var Pbat_dis[7] (value 0.0)>,
 8: <gurobi.Var Pbat_dis[8] (value 0.0)>,
 9: <gurobi.Var Pbat_dis[9] (value 0.0)>,
 10: <gurobi.Var Pbat_dis[10] (value 0.0)>,
 11: <gurobi.Var Pbat_dis[11] (value 0.0)>,
 12: <gurobi.Var Pbat_dis[12] (value 0.0)>,
 13: <gurobi.Var Pbat_dis[13] (value 0.0)>,
 14: <gurobi.Var Pbat_dis[14] (value 0.0)>,
 15: <gurobi.Var Pbat_dis[15] (value 0.0)>,
 16: <gurobi.Var Pbat_dis[16] (value 0.0)>,
 17: <gurobi.Var Pbat_dis[17] (value 0.0)>,
 18: <gurobi.Var Pbat_dis[18] (value 0.0)>,
 19: <gurobi.Var Pbat_dis[19] (value 0.0)>,
 20: <gurobi.Var Pbat_dis[20] (value 0.0)>,
 21: <gurobi.Var Pbat_dis[21] (value 0.0)>,
 22: <gurobi.Var Pbat_dis[22] (value